In [ ]:
import sys
!"{sys.executable}" -m pip install --upgrade pyarrow

In [ ]:
import pandas as pd
import pyarrow
import sys

print("Python:", sys.executable)
print("pandas:", pd.__version__)
print("pyarrow:", pyarrow.__version__)